In [1]:
"""
Created on Sun Dec 01 22:55:07 2022 

@author: Hualong Liu, Shweta Meena
"""

'\nCreated on Sun Dec 01 22:55:07 2022 \n\n@author: Hualong Liu, Shweta Meena\n'

In [ ]:
#------------------------Definition of variables ----------------------
import cvxpy as cp
import numpy as np
import cmath
from numpy.linalg import inv
import math 
from mpmath import sec
import time
import matplotlib.pyplot as plt
start_time = time.time()


# Base parameters
factor = 1
T = 24
NODE = 4
TypicalDay = 4
PROJECT_LIFETIME = 15
M = 10000
M1 = 10000
N = 3
# DG
POWER_RATING_DG = 2500 # KW
CAPITAL_COST_DG = 800 # $/KW please see table 3 for this parameter
FIXED_ONM_COST_DG = 25 # ($/kw-year)
GEN_COST_DG = 0.06675*factor # $/kwh, on hourly basis kw = kwh
DG_INSTALL_FIXCOST = 800 # cost of installing DER at a node 


# Battery
batChaEff = 0.85 # batteryChargingEfficiency
batDisEff = 0.85 # batteryDischargingEfficiency
SOC_min = 0.2
SOC_max = 0.99
Large_Bat = 10000 # Used to prevent the battery from charging and discharging simultaneously
Batt_rate = 4 # Time to charge/discharge when charging/discharging with Pmax 
BATTERY_CAPITAL_COST = 320 # $/kW-hr 
BATTERY_ONM_COST = 8 # $/(kWh-year) # operation and maintennace cost 
BATTERY_INSTALL_FIXCOST = 800 #1280  # installation cost at node inpependent of the capacity 
Self_Leakage_rate = 0 #0.01/100


# PV 
PV_CAPITAL_COST = 2000 # $/kw
PV_ONM_COST = 20 # $/(year-kw)
PV_INSTALL_FIXCOST = 800 #2000

# Grid 
A_LARGE_NUMBER = 1000 # max exchange power at PCC
CURTAIL_MAX = 100
Purchased_Electricity_Rate = 0.08*factor #$/kWh 
Exported_Electricity_Rate = 0.04*factor #$/kWh
#Operation_Cost_ICE2 = 0.08 # $/kWh
curtailedLoad_Cost = 0.08*1.2*factor # $/kW



# # This is used for calculating the annual cost, please see the first line of Eq1
Nominal_Discount_Rate = 6.55/100
Expected_Inflation_Rate = 3/100
Real_Discount_Rate = (Nominal_Discount_Rate - Expected_Inflation_Rate) / (1 + Expected_Inflation_Rate)
Annual_Recovery_Rate = Real_Discount_Rate * (1 + Real_Discount_Rate) ** PROJECT_LIFETIME \
/ ((1 + Real_Discount_Rate) ** PROJECT_LIFETIME - 1)


# Load Profiles 
electricalLoad_Summer_Node1 = np.array([20, 20, 20, 20, 40, 140, 200, 420, 420, 420, 420, 420, \
    420, 420, 420, 420, 300, 200, 200, 180, 180, 60, 20, 20]) # kW
electricalLoad_Spring_Node1 = 0.6 * electricalLoad_Summer_Node1.astype(float)
electricalLoad_Fall_Node1 = 0.8 * electricalLoad_Summer_Node1.astype(float)
electricalLoad_Winter_Node1 = 0.9 * electricalLoad_Summer_Node1.astype(float)
electricalLoad_Node1 = np.array([electricalLoad_Spring_Node1, electricalLoad_Summer_Node1, \
                                        electricalLoad_Fall_Node1, electricalLoad_Winter_Node1])

electricalLoad_Summer_Node2 = np.array([280, 240, 200, 220, 280, 440, 440, 400, 400, 440, 460, \
    460, 460, 460, 440, 300, 440, 440, 600, 640, 580, 580, 380, 380])
electricalLoad_Spring_Node2 = 0.6 * electricalLoad_Summer_Node2.astype(float)
electricalLoad_Fall_Node2 = 0.8 * electricalLoad_Summer_Node2.astype(float)
electricalLoad_Winter_Node2 = 0.9 * electricalLoad_Summer_Node2.astype(float)
electricalLoad_Node2 = np.array([electricalLoad_Spring_Node2, electricalLoad_Summer_Node2, \
                                       electricalLoad_Fall_Node2, electricalLoad_Winter_Node2])

electricalLoad_Summer_Node3 = np.array([1000, 1000, 1000, 1000, 1400, 2000, 2500, 2500, 2500, \
    2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2000, 1000])
electricalLoad_Spring_Node3 = 0.6 * electricalLoad_Summer_Node3.astype(float)
electricalLoad_Fall_Node3 = 0.8 * electricalLoad_Summer_Node3.astype(float)
electricalLoad_Winter_Node3 = 0.9 * electricalLoad_Summer_Node3.astype(float)
electricalLoad_Node3 =np.array([electricalLoad_Spring_Node3, electricalLoad_Summer_Node3, \
                                        electricalLoad_Fall_Node3, electricalLoad_Winter_Node3])
# np.ones((1,T))*1000
electricalLoad_Summer_Node4 = np.array([50, 50, 50, 50, 100, 300, 500, 900, 900, 900, 900, 900, \
    900, 900, 900, 900, 800, 500, 500, 400, 400, 100, 50, 50])
electricalLoad_Spring_Node4 = 0.6 * electricalLoad_Summer_Node4.astype(float)
electricalLoad_Fall_Node4 = 0.8 * electricalLoad_Summer_Node4.astype(float)
electricalLoad_Winter_Node4 = 0.9 * electricalLoad_Summer_Node4.astype(float)
electricalLoad_Node4 =  np.array([electricalLoad_Spring_Node4, electricalLoad_Summer_Node4, \
                                        electricalLoad_Fall_Node4, electricalLoad_Winter_Node4])


# Weather data 
Irradiance_Spring = np.array([0, 0, 0, 0, 0, 0, 0, 0.05, 0.25, 0.4, 0.45, 0.61, 0.58, 0.59, 0.60, 0.4, 0.3, 0.2, 0, \
    0, 0, 0, 0, 0]) # kW/m^2
Irradiance_Summer = np.array([0, 0, 0, 0, 0, 0, 0.12, 0.18, 0.32, 0.5, 0.52, 0.7, 0.8, 0.78, 0.77, 0.61, 0.3, \
    0.27, 0.05, 0, 0, 0, 0, 0])
Irradiance_Fall = 0.85 * Irradiance_Spring.astype(float)
Irradiance_Winter = 0.65 * Irradiance_Spring.astype(float)

# Temperature_Spring = np.array([5.2, 5.3, 5.2, 5.2, 5.3, 5.3, 5.3, 8, 11, 13, 14, 14.1, 14, 14.2, 14.1, 14, 14.2, \
#     14, 12, 12.2, 12, 11, 9, 6]) #Celsius degrees
# Temperature_Summer = 4.6 * Temperature_Spring.astype(float)
# Temperature_Fall = 2.88 * Temperature_Spring.astype(float)
# Temperature_Winter = 0.4 * Temperature_Spring.astype(float)


Temperature_Spring = np.array([5.2, 5.3, 5.2, 5.2, 5.3, 5.3, 5.3, 8, 11, 13, 14, 14.1, 14, 14.2, 14.1, 14, 14.2, \
    14, 12, 12.2, 12, 11, 9, 6]) #Celsius degrees
Temperature_Summer = 2.2 * Temperature_Spring.astype(float)
Temperature_Fall = 1.6 * Temperature_Spring.astype(float)
Temperature_Winter = 0.5 * Temperature_Spring.astype(float)


PV_STC = 1.0 # kW
PV_Output_Spring = np.zeros(T)
PV_Output_Summer = np.zeros(T)
PV_Output_Fall = np.zeros(T)
PV_Output_Winter = np.zeros(T)
for i in range(T):
    PV_Output_Spring[i] = PV_STC \
    * Irradiance_Spring[i] / 1 * (1 - 0.0045 * (Temperature_Spring[i] + 273.15 - 298))
    PV_Output_Summer[i] = PV_STC \
    * Irradiance_Summer[i] / 1 * (1 - 0.0045 * (Temperature_Summer[i] + 273.15 - 298))
    PV_Output_Fall[i] = PV_STC \
    * Irradiance_Fall[i] / 1 * (1 - 0.0045 * (Temperature_Fall[i] + 273.15 - 298))
    PV_Output_Winter[i] = PV_STC \
    * Irradiance_Winter[i] / 1 * (1 - 0.0045 * (Temperature_Winter[i] + 273.15 - 298))

#PV_Output = np.array([PV_Output_Spring, PV_Output_Summer, PV_Output_Fall, PV_Output_Winter])
PV_Output = np.array([PV_Output_Spring/max(PV_Output_Spring), PV_Output_Summer/max(PV_Output_Summer), PV_Output_Fall/max(PV_Output_Fall), PV_Output_Winter/max(PV_Output_Winter)])

# Power flow 
##############################
cos_phi = 0.8 # 1 # Power factor 
Sbase = 1 # MVA System base power for pu impedance given, common choice 100 MVA
Vbase = 12   # System Volatge in kv
Zbase =  Vbase**(2)/Sbase
R_pu = 1.678*40*1e-4/Zbase # #64*10e-10 #1.678*40*1e-4 5.48*1e-4 
X_pu = 1.678*1e-4/Zbase #  #1.4*10e-10
Ibase = Sbase/(Vbase*math.sqrt(3)) # kA

# Voltage 
Vo = 12 # Volatge in kV
V_max = 1.05*Vo/Vbase
V_min = 0.90*Vo/Vbase
Pi = math.pi
Theta_max = 10*Pi/180  # maximum angle for bus voltages 
Theta_min = -11*Pi/180 # minimum angle for bus voltages 

# Cable current limits
Ir_limit = np.empty((NODE+1,NODE+1)) # Max expected value of real current of line L
Ii_limit = np.empty((NODE+1,NODE+1))  # Max expected value of imaginary current of line L
I_limit = np.empty((NODE,NODE))  # Current carrying capacity of line L 
Z_cable = np.empty(NODE+1,complex) 
Y_cable = np.empty(NODE+1,complex) 
# Y_real = np.empty(NODE+1)
# Y_img =  np.empty(NODE+1)

line_len = [1200,1800,900,1800,1200]  # line joining node 1 to node 2 is the line zero

for i in range(NODE+1):
    for j in range(NODE+1):
        Ir_limit[i,j] = 0.4*Ibase 
        Ii_limit[i,j] = 0.4*Ibase
#        
for i in range(NODE+1):
    Z_cable[i] = complex(R_pu*line_len[i]*Zbase, X_pu*line_len[i]*Zbase)
#     Y_real[i] = (1/(Z_cable[i])).real
#     Y_img[i] =  (1/(Z_cable[i])).imag
    Y_cable[i] = (1/(Z_cable[i]))
    

Y_bus = np.zeros((NODE+1,NODE+1),complex)
for i in range(NODE+1):
    if(i!= NODE):
        Y_bus[i,i] = Y_cable[i] + Y_cable[i-1]
        Y_bus[i,i+1] = -Y_cable[i]
        Y_bus[i,i-1]= -Y_cable[i-1]
    else:
        Y_bus[i,i] = Y_cable[i] + Y_cable[i-1]
        Y_bus[i,0] = -Y_cable[i]
        Y_bus[i,i-1]= -Y_cable[i-1]
        
Ybus_real = Y_bus.real
Ybus_img = Y_bus.imag

z_real = np.empty((NODE,NODE),complex) # Thev impedance real part 
z_img = np.empty((NODE,NODE),complex)  # Thev impedance imag part 


# Ybus_real = np.empty((NODE,NODE),complex)
# Ybus_img = np.empty((NODE,NODE),complex)

Z_line = np.empty((NODE+1,NODE+1), complex) 
r_line = np.empty((NODE+1,NODE+1)) 
x_line = np.empty((NODE+1,NODE+1))
for i in range(NODE+1):
    if(i!= NODE):
        Z_line[i,i] = 0
        Z_line[i,i+1] = Z_cable[i]
        Z_line[i,i-1]= Z_cable[i-1]
    else: 
        Z_line[i,i] = 0
        Z_line[i,0] = Z_cable[i]
        Z_line[i,i-1]= Z_cable[i-1]

r_line = Z_line.real
x_line = Z_line.imag

Z_bus = inv(Y_bus[0:4,0:4]) 
z_real = Z_bus.real
z_img = Z_bus.imag


##########################################

In [ ]:
Annual_Recovery_Rate = Real_Discount_Rate * (1 + Real_Discount_Rate) ** PROJECT_LIFETIME \
/ ((1 + Real_Discount_Rate) ** PROJECT_LIFETIME - 1)
Annual_Recovery_Rate*15

In [ ]:
for i in range(NODE):
    display(np.linalg.norm(Z_bus[i,:]/Zbase))

In [ ]:
Ir_limit[0,0]

In [ ]:
# Verifying the correctness of linear power flow 
2*2*1/Ibase*math.sqrt(Ir_limit[0,0]**2 + Ii_limit[0,0]**2)*4*0.104
# 

In [ ]:
#------------------------Definition of variables----------------------
batteryCAP = cp.Variable(NODE,"BatteryCap") # power capacity and energy capacity of the battery 
batteryPow_MAX = cp.Variable(NODE, "BattRating")
#batteryPow_MAX = np.array([400, 400, 400 , 400])
PVCAP= cp.Variable(NODE, "PVCap") # pv power capacity
DGNumber = cp.Variable(NODE, integer=True, name = "DGNumber") # ice power capacity
batteryDecision = cp.Variable(NODE, boolean=True, name = "BattDecision")
PVDecision = cp.Variable(NODE, boolean=True, name = "PVDecision")
DGDecision = cp.Variable(NODE, boolean=True, name = "DGDecision")
# '''
# The number of DG ICE is a discrete variable and does not require the binary decision variable. 
# Please refer to Equation 1
# '''


# injectedPowerNode5_1 = cp.Variable((TypicalDay, T), "ActivePowerNode_5_1")
# injectedPowerNode1_2 = cp.Variable((TypicalDay, T), "ActivePowerNode_1_2")
# injectedReactivePowerNode5_1 = cp.Variable((TypicalDay, T), "ReactivePowerNode_5_1")
# injectedReactivePowerNode1_2 = cp.Variable((TypicalDay, T), "ReactivePowerNode_1_2")
generationPVNode1 = cp.Variable((TypicalDay, T), "GenPV_Node1")
generationDGNode1 = cp.Variable((TypicalDay, T), "GenDG_Node1") 
eleLoaCurNode1 = cp.Variable((TypicalDay, T), "LoadCur_Node1") # electricalLoadCurtailed
batDisPowNode1 = cp.Variable((TypicalDay, T), "BatDisPow_Node1") # batteryDischargePower
batChaPowNode1 = cp.Variable((TypicalDay, T), "BatCharPow_Node1") # batteryChargePower

# injectedPowerNode2_3 = cp.Variable((TypicalDay, T), "ActivePowerNode_2_3")
# injectedReactivePowerNode2_3 = cp.Variable((TypicalDay, T), "ReactivePowerNode_2_3")
generationPVNode2 = cp.Variable((TypicalDay, T), "GenPV_Node2")
generationDGNode2 = cp.Variable((TypicalDay, T), "GenDG_Node2") 
eleLoaCurNode2 = cp.Variable((TypicalDay, T),"LoadCur_Node2") # electricalLoadCurtailed
batDisPowNode2 = cp.Variable((TypicalDay, T),"BatDisPow_Node2") # batteryDischargePower
batChaPowNode2 = cp.Variable((TypicalDay, T),"BatCharPow_Node2") # batteryChargePower

# injectedPowerNode3_4 = cp.Variable((TypicalDay, T), "ActivePowerNode_3_4")
# injectedReactivePowerNode3_4 = cp.Variable((TypicalDay, T), "ReactivePowerNode_3_4")
generationPVNode3 = cp.Variable((TypicalDay, T),"GenPV_Node3")
generationDGNode3 = cp.Variable((TypicalDay, T),"GenDG_Node3") 
eleLoaCurNode3 = cp.Variable((TypicalDay, T), "LoadCur_Node3") # electricalLoadCurtailed
batDisPowNode3 = cp.Variable((TypicalDay, T), "BatDisPow_Node3") # batteryDischargePower
batChaPowNode3 = cp.Variable((TypicalDay, T),"BatCharPow_Node3") # batteryChargePower

# injectedPowerNode4_5 = cp.Variable((TypicalDay, T), "ActivePowerNode_4_5")
# injectedReactivePowerNode4_5 = cp.Variable((TypicalDay, T), "ReactivePowerNode_4_5")
generationPVNode4 = cp.Variable((TypicalDay, T), "GenPV_Node4")
generationDGNode4 = cp.Variable((TypicalDay, T),"GenDG_Node4") 
eleLoaCurNode4 = cp.Variable((TypicalDay, T),"LoadCur_Node4") # electricalLoadCurtailed
batDisPowNode4 = cp.Variable((TypicalDay, T),"BatDisPow_Node4") # batteryDischargePower
batChaPowNode4 = cp.Variable((TypicalDay, T),"BatCharPow_Node4") # batteryChargePower


batEneIni = cp.Variable(NODE) # BatteryEnergyInitial 
batteryEnergyNode1 = cp.Variable((TypicalDay, T), "BattEnergy_Node1")   
batteryEnergyNode2 = cp.Variable((TypicalDay, T), "BattEnergy_Node2")
batteryEnergyNode3 = cp.Variable((TypicalDay, T), "BattEnergy_Node3")
batteryEnergyNode4 = cp.Variable((TypicalDay, T), "BattEnergy_Node4")

batDisFlaNode1 = cp.Variable((TypicalDay, T), boolean=True, name = "BatDisFlag_Node1") # batteryDischargeFlagNode1 
batDisFlaNode2 = cp.Variable((TypicalDay, T), boolean=True, name = "BatDisFlag_Node2")  
batDisFlaNode3 = cp.Variable((TypicalDay, T), boolean=True, name = "BatDisFlag_Node3") 
batDisFlaNode4 = cp.Variable((TypicalDay, T), boolean=True, name = "BatDisFlag_Node4") 


purchasedElectricity = cp.Variable((TypicalDay, T), "Elec_pur") # at node 5 (PCC)  
exportedElectricity = cp.Variable((TypicalDay, T), "Elec_expr")
purDec = cp.Variable((TypicalDay, T), boolean=True, name = "PurDecision") # purchaseSell_Decision


# Power flow
# Active, Reactive power at nodes 
injectedActivePowerNode = {}
injectedReactivePowerNode = {}
for i in range(NODE+1):
    injectedActivePowerNode[i] = cp.Variable((TypicalDay, T), name = "InjectedActivePowerNode")
    injectedReactivePowerNode[i] = cp.Variable((TypicalDay, T), name = "InjectedReactivePowerNode")
    
# injectedActivePowerNode = np.empty(Node,TypicalDay, T)
# injectedActivePowerNode[:,:,:] = cp.variable()

# Voltage at Nodes
VoltageRealPartNode = {}
VoltageImagPartNode = {}
for i in range(NODE+1):
    VoltageRealPartNode[i] = cp.Variable((TypicalDay, T), "NodeVoltageRealPart")
    VoltageImagPartNode[i] = cp.Variable((TypicalDay, T), "NodeVoltageImagPart")
    
#line current flows
CurrentRealPartNode = np.zeros((NODE+1,NODE+1),cp.Variable)
CurrentImagPartNode = np.zeros((NODE+1,NODE+1),cp.Variable)

for i in range(NODE+1):
    for j in range(NODE+1):
        CurrentRealPartNode[i,j] = cp.Variable((TypicalDay, T), "LineCurrentRealPart")
        CurrentImagPartNode[i,j] = cp.Variable((TypicalDay, T), "LineCurrentImagPart")

Current_Sq_RealPartNode = np.zeros((NODE+1,NODE+1),cp.Variable)
Current_Sq_ImagPartNode = np.zeros((NODE+1,NODE+1),cp.Variable)

for i in range(NODE+1):
    for j in range(NODE+1):
        Current_Sq_RealPartNode[i,j] = cp.Variable((TypicalDay, T), "CurrentSqRealPart")
        Current_Sq_ImagPartNode[i,j] = cp.Variable((TypicalDay, T), "CurrentSqImagPart")

Ploss = cp.Variable((TypicalDay, T), "Ploss")
Qloss = cp.Variable((TypicalDay, T), "Qloss") 

# CurrentImagPartNode = {}
# for i in range(NODE):
#     VoltageRealPartNode[i] = cp.Variable((TypicalDay, T))
#     VoltageImagPartNode[i] = cp.Variable((TypicalDay, T))


In [ ]:
#------------------------Definition of constraints------------------------
constraints = []

# Active Power injections at nodes[1,4] (Gen - load)

for i in range(TypicalDay):
    for j in range(T):
        constraints += [generationPVNode1[i, j] + generationDGNode1[i, j] - (electricalLoad_Node1[i, j] - \
        eleLoaCurNode1[i, j]) + batDisPowNode1[i, j] - batChaPowNode1[i, j]  == injectedActivePowerNode[0][i,j]]
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [generationPVNode2[i, j] + generationDGNode2[i, j] - (electricalLoad_Node2[i, j] - \
        eleLoaCurNode2[i, j]) + batDisPowNode2[i, j] - batChaPowNode2[i, j] == injectedActivePowerNode[1][i,j]]
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [generationPVNode3[i, j] + generationDGNode3[i, j] - (electricalLoad_Node3[i, j] - \
        eleLoaCurNode3[i, j]) + batDisPowNode3[i, j] - batChaPowNode3[i, j] == injectedActivePowerNode[2][i,j]]
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [generationPVNode4[i, j] + generationDGNode4[i, j] - (electricalLoad_Node4[i, j] - \
        eleLoaCurNode4[i, j]) + batDisPowNode4[i, j] - batChaPowNode4[i, j] == injectedActivePowerNode[3][i,j]]
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [purchasedElectricity[i, j] - exportedElectricity[i, j] == injectedActivePowerNode[4][i,j]]

# Reactive Power Injections        
for i in range(NODE):
    constraints += [injectedReactivePowerNode[i] == injectedActivePowerNode[i]*math.tan(math.acos(cos_phi))] 

# Linearized Voltage equations (in p.u.)
for i in range(NODE): 
    constraints += [VoltageRealPartNode[i] == (Vo/Vbase) + 1/(Vo/Vbase)*(sum(z_real[i,j]*injectedActivePowerNode[j] + \
                                                          z_img[i,j]*injectedReactivePowerNode[j] for j in range(NODE)))/(Zbase*Sbase*1000)]
    constraints += [VoltageImagPartNode[i] ==  1/(Vo/Vbase)*(sum(z_img[i,j]*injectedActivePowerNode[j] - \
                                                          z_real[i,j]*injectedReactivePowerNode[j] for j in range(NODE)))/(Zbase*Sbase*1000)]
constraints += [VoltageRealPartNode[NODE] == Vo/Vbase, VoltageImagPartNode[NODE] == 0]

# Voltage upper and lower bound constarints 
K = ((math.sin(Theta_max)-math.sin(Theta_min))/(math.cos(Theta_max)-math.cos(Theta_min)))

for i in range(NODE):
    constraints += [VoltageImagPartNode[i] <= K*(VoltageRealPartNode[i]-V_min*sec((Theta_max-Theta_min)/2)*math.cos(Theta_min)) \
                 + V_min*sec((Theta_max-Theta_min)/2)*math.sin(Theta_min)]
    constraints += [VoltageImagPartNode[i]<= (math.sin(Theta_max)/(math.cos(Theta_max)-1))*(VoltageRealPartNode[i] - V_max)]
    constraints += [VoltageImagPartNode[i]<= (-math.sin(Theta_min)/(math.cos(Theta_min)-1))*(VoltageRealPartNode[i] - V_max)]
    constraints += [VoltageRealPartNode[i]*math.tan(Theta_min)<=VoltageImagPartNode[i],\
                     VoltageImagPartNode[i]<=VoltageRealPartNode[i]*math.tan(Theta_max)]
# losseless design 

#constraints += [0 == sum(injectedActivePowerNode[i] for i in range(NODE))]
#constraints += [0 == sum(injectedReactivePowerNode[i] for i in range(NODE))]


# Cable current calculations (A)
for i in range(NODE+1):
     for j in range(NODE+1):
        constraints += [CurrentRealPartNode[i,j] == (-Ybus_real[i,j]*(VoltageRealPartNode[i]-VoltageRealPartNode[j])*Vbase + \
                        Ybus_img[i,j]*(VoltageImagPartNode[i]-VoltageImagPartNode[j])*Vbase)*1000]
        constraints += [CurrentImagPartNode[i,j] == (-Ybus_img[i,j]*(VoltageRealPartNode[i]-VoltageRealPartNode[j])*Vbase - \
                        Ybus_real[i,j]*(VoltageImagPartNode[i]-VoltageImagPartNode[j])*Vbase)*1000]
        
# Cable current constraints 
for i in range(NODE+1):
     for j in range(NODE+1):
        constraints += [CurrentRealPartNode[i,j]<=Ir_limit[i,j]*1000, CurrentRealPartNode[i,j]>=-Ir_limit[i,j]*1000]
        constraints += [CurrentImagPartNode[i,j]<=Ii_limit[i,j]*1000, CurrentImagPartNode[i,j]>=-Ii_limit[i,j]*1000]        
 
 



# Current_Sq_ImagPartNode[i,j]+Current_Sq_RealPartNode[i,j]

# # Cable current constraints 
# for i in range(NODE+1):
#      for j in range(NODE+1):
#         constraints += [Current_Sq_ImagPartNode[i,j]+Current_Sq_RealPartNode[i,j]<=Ir_limit[i,j]*1000]
#         constraints += [Current_Sq_ImagPartNode[i,j]+Current_Sq_RealPartNode[i,j]=-Ii_limit[i,j]*1000]



Nv = 30

delta_Ir = np.empty((NODE+1,NODE+1))  # in A
delta_Ii =  np.empty((NODE+1,NODE+1))



# Cable current square approximation  # in MA 
for i in range(NODE+1):
     for j in range(NODE+1):
        delta_Ir[i,j] = Ir_limit[i,j]*1000/Nv # in A 
        delta_Ii[i,j] = Ii_limit[i,j]*1000/Nv
        for v in range(Nv):
            constraints +=[Current_Sq_RealPartNode[i,j]>=(v*delta_Ir[i,j])**2 + \
            (2*v-1)*delta_Ir[i,j]*(CurrentRealPartNode[i,j]-v*delta_Ir[i,j])]
            constraints +=[Current_Sq_RealPartNode[i,j]>=(v*delta_Ir[i,j])**2 - \
            (2*v-1)*delta_Ir[i,j]*(CurrentRealPartNode[i,j]+v*delta_Ir[i,j])]
            constraints +=[Current_Sq_ImagPartNode[i,j]>=(v*delta_Ii[i,j])**2 + \
            (2*v-1)*delta_Ii[i,j]*(CurrentImagPartNode[i,j]-v*delta_Ii[i,j])]
            constraints +=[Current_Sq_ImagPartNode[i,j]>=(v*delta_Ii[i,j])**2 - \
            (2*v-1)*delta_Ii[i,j]*(CurrentImagPartNode[i,j]+v*delta_Ii[i,j])]

            
# Loss constraint (in kW)
constraints += [Ploss == 0.5*sum(r_line[i,j]*(Current_Sq_ImagPartNode[i,j]+Current_Sq_RealPartNode[i,j]) \
                                for i in range(NODE+1) for j in range(NODE+1))/1000]
constraints += [Qloss == 0.5*sum(x_line[i,j]*(Current_Sq_ImagPartNode[i,j]+Current_Sq_RealPartNode[i,j]) \
                                for i in range(NODE+1) for j in range(NODE+1))/1000]  

# constraints += [injectedActivePowerNode[4] == Ploss]
# constraints += [injectedReactivePowerNode[4] == Qloss]


# constraints += [injectedActivePowerNode[4] == Ploss]
# constraints += [injectedReactivePowerNode[4] == Qloss]

constraints += [Ploss == sum(injectedActivePowerNode[i] for i in range(NODE+1))]
constraints += [Qloss == sum(injectedReactivePowerNode[i] for i in range(NODE+1))]


# ******************* NEED TO FIX GRID IMPORT EXPORT *********************************
#constraints for purchased and exported
# for i in range(TypicalDay):
#     for j in range(T):
#         constraints += [purchasedElectricity[i, j] - exportedElectricity[i, j] == injectedActivePowerNode[4]]

for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= purchasedElectricity[i, j], purchasedElectricity[i, j] <= purDec[i, j] * A_LARGE_NUMBER, \
                        0 <= exportedElectricity[i, j], exportedElectricity[i, j] <= \
                        (1 - purDec[i, j]) * A_LARGE_NUMBER]
        
# generation and storage constraints
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= generationDGNode1[i, j],  generationDGNode1[i, j] <= DGNumber[0] * POWER_RATING_DG]
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= generationDGNode2[i, j],  generationDGNode2[i, j] <= DGNumber[1] * POWER_RATING_DG]
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= generationDGNode3[i, j],  generationDGNode3[i, j] <= DGNumber[2] * POWER_RATING_DG]
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= generationDGNode4[i, j],  generationDGNode4[i, j] <= DGNumber[3] * POWER_RATING_DG]


for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= generationPVNode1[i, j],  generationPVNode1[i, j] <= PVCAP[0] * PV_Output[i, j]] 
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= generationPVNode2[i, j],  generationPVNode2[i, j] <= PVCAP[1] * PV_Output[i, j]] 
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= generationPVNode3[i, j],  generationPVNode3[i, j] <= PVCAP[2] * PV_Output[i, j]] 
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= generationPVNode4[i, j],  generationPVNode4[i, j] <= PVCAP[3] * PV_Output[i, j]] 

# battery energy constraints

      

#Shweta
for i in range(NODE):
        constraints += [batEneIni[i] == 0.8 * batteryCAP[i]] 

for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= batDisPowNode1[i, j],  batDisPowNode1[i, j] <= batDisFlaNode1[i, j] * \
         Large_Bat* batDisEff, \
        0 <= batChaPowNode1[i, j],  batChaPowNode1[i, j] <= (1 - batDisFlaNode1[i, j]) * \
        Large_Bat/ batChaEff] 

        constraints += [0 <= batDisPowNode2[i, j],  batDisPowNode2[i, j] <= batDisFlaNode2[i, j] * \
         Large_Bat * batDisEff, \
        0 <= batChaPowNode2[i, j],  batChaPowNode2[i, j] <= (1 - batDisFlaNode2[i, j]) * \
        Large_Bat/ batChaEff] 

        constraints += [0 <= batDisPowNode3[i, j],  batDisPowNode3[i, j] <= batDisFlaNode3[i, j] * \
         Large_Bat * batDisEff, \
        0 <= batChaPowNode3[i, j],  batChaPowNode3[i, j] <= (1 - batDisFlaNode3[i, j]) * \
         Large_Bat / batChaEff] 

        constraints += [0 <= batDisPowNode4[i, j],  batDisPowNode4[i, j] <= batDisFlaNode4[i, j] * \
        Large_Bat * batDisEff, \
        0 <= batChaPowNode4[i, j],  batChaPowNode4[i, j] <= (1 - batDisFlaNode4[i, j]) * \
         Large_Bat / batChaEff] 
        
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= batDisPowNode1[i, j],  batDisPowNode1[i, j] <= batteryPow_MAX[0] * batDisEff, \
        0 <= batChaPowNode1[i, j],  batChaPowNode1[i, j] <= batteryPow_MAX[0] / batChaEff] 

        constraints += [0 <= batDisPowNode2[i, j],  batDisPowNode2[i, j] <= batteryPow_MAX[1] * batDisEff, \
        0 <= batChaPowNode2[i, j],  batChaPowNode2[i, j] <= batteryPow_MAX[1] / batChaEff] 

        constraints += [0 <= batDisPowNode3[i, j],  batDisPowNode3[i, j] <= batteryPow_MAX[2] * batDisEff, \
        0 <= batChaPowNode3[i, j],  batChaPowNode3[i, j] <= batteryPow_MAX[2] / batChaEff] 

        constraints += [0 <= batDisPowNode4[i, j],  batDisPowNode4[i, j] <= batteryPow_MAX[3] * batDisEff, \
        0 <= batChaPowNode4[i, j],  batChaPowNode4[i, j] <= batteryPow_MAX[3] / batChaEff]

        
for i in range(NODE):        
    constraints += [batteryPow_MAX[i] == batteryCAP[i]/Batt_rate]



 
    
#Shweta      
for i in range(TypicalDay):
    constraints += [ batteryEnergyNode1[i, 0] == (1 - Self_Leakage_rate) * batEneIni[0] + batChaEff * batChaPowNode1[i, 0] -\
                    batDisPowNode1[i, 0] / batDisEff] 
    constraints += [ batteryEnergyNode2[i, 0] == (1 - Self_Leakage_rate) * batEneIni[1] + batChaEff * batChaPowNode2[i, 0] -\
                    batDisPowNode2[i, 0] / batDisEff]
    constraints += [ batteryEnergyNode3[i, 0] == (1 - Self_Leakage_rate) * batEneIni[2] + batChaEff * batChaPowNode3[i, 0] -\
                    batDisPowNode3[i, 0] / batDisEff]              
    constraints += [ batteryEnergyNode4[i, 0] == (1 - Self_Leakage_rate) * batEneIni[3] + batChaEff * batChaPowNode4[i, 0] -\
                    batDisPowNode4[i, 0] / batDisEff]                      

for i in range(TypicalDay):
    for j in range(1,T):
        constraints += [ batteryEnergyNode1[i, j] == (1 - Self_Leakage_rate) * batteryEnergyNode1[i, j-1] + \
                        batChaEff * batChaPowNode1[i, j] - batDisPowNode1[i, j] / batDisEff] 
        constraints += [ batteryEnergyNode2[i, j] == (1 - Self_Leakage_rate) *  batteryEnergyNode2[i, j-1] + \
                        batChaEff * batChaPowNode2[i, j] - batDisPowNode2[i, j] / batDisEff]
        constraints += [ batteryEnergyNode3[i, j] == (1 - Self_Leakage_rate) * batteryEnergyNode3[i, j-1] + \
                        batChaEff * batChaPowNode3[i, j] - batDisPowNode3[i, j] / batDisEff]              
        constraints += [ batteryEnergyNode4[i, j] == (1 - Self_Leakage_rate) * batteryEnergyNode4[i, j-1] + \
                        batChaEff * batChaPowNode4[i, j] - batDisPowNode4[i, j] / batDisEff]  
    
       
for i in range(TypicalDay):
    for j in range(1,T):
        constraints += [SOC_min * batteryCAP[0] <= batteryEnergyNode1[i,j], batteryEnergyNode1[i,j] <= SOC_max * batteryCAP[0],\
        SOC_min * batteryCAP[1] <= batteryEnergyNode2[i,j], \
                        batteryEnergyNode2[i,j] <= SOC_max  *batteryCAP[1], \
        SOC_min * batteryCAP[2] <= batteryEnergyNode3[i,j], \
                        batteryEnergyNode3[i,j] <= SOC_max  *batteryCAP[2], \
        SOC_min * batteryCAP[3] <= batteryEnergyNode4[i,j], \
                        batteryEnergyNode4[i,j] <= SOC_max  *batteryCAP[3]]
            
    
# SOC at start of the day = SOC at the end of the day     
for i in range(TypicalDay):
    constraints += [batteryEnergyNode1[i, T-1] == batEneIni[0], \
    batteryEnergyNode2[i, T-1] == batEneIni[1], \
    batteryEnergyNode3[i, T-1] == batEneIni[2], \
    batteryEnergyNode4[i, T-1] == batEneIni[3]] 
    
# Load curtailement limit 
for i in range(TypicalDay):
    for j in range(T):
        constraints += [0 <= eleLoaCurNode1[i, j], eleLoaCurNode1[i, j] <= CURTAIL_MAX, \
                        0 <= eleLoaCurNode2[i, j], eleLoaCurNode2[i, j] <= CURTAIL_MAX, \
                        0 <= eleLoaCurNode3[i, j], eleLoaCurNode3[i, j] <= CURTAIL_MAX, \
                        0 <= eleLoaCurNode4[i, j], eleLoaCurNode4[i, j] <= CURTAIL_MAX] 
        
# Limit on total installed capacity of a particular DG    
# other meaningful constraints 
#**********************ADD MORE************************
for i in range(NODE):
    constraints += [PVCAP[i] <=PVDecision[i]*M1]
    constraints += [batteryCAP[i] <=batteryDecision[i]*M]
    constraints += [DGNumber[i] <= DGDecision[i]*M]
    
#constraints += [cp.sum(DGDecision)<=2]    
#constraints += [cp.sum(PVDecision) <=2]
# constraints += [cp.sum(batteryDecision) <=2]
# constraints += [DGNumber[0] ==1]
# constraints += [PVDecision[1] ==1]
# constraints += [PVDecision[2] ==1]
# constraints += [batteryDecision[3] ==1]
# constraints += [cp.sum(PVCAP) <= 2000] # To ensure that all kinds of equipment are used
# constraints += [cp.sum(batteryCAP) <= 2000]    

In [ ]:
# ------------------------Objective------------------------
#acquisitionUnitCost_DG = np.array([POWER_RATING_DG * CAPITAL_COST_DG * Annual_Recovery_Rate] * NODE)
# DG
acquisitionUnitCost_DG = np.array([POWER_RATING_DG * (CAPITAL_COST_DG*Annual_Recovery_Rate + FIXED_ONM_COST_DG )] * NODE)
installCost_DG = np.array([DG_INSTALL_FIXCOST]*NODE)
objective = cp.sum(cp.multiply(DGNumber, acquisitionUnitCost_DG)) + cp.sum(cp.multiply(installCost_DG, DGDecision))



# Battery 
for i in range(NODE):
    objective += ((BATTERY_CAPITAL_COST* Annual_Recovery_Rate + BATTERY_ONM_COST) * batteryCAP[i]) + \
                    BATTERY_INSTALL_FIXCOST * batteryDecision[i] 
    

    
# PV     
for i in range(NODE):
    objective += ( (PV_CAPITAL_COST* Annual_Recovery_Rate + PV_ONM_COST)* PVCAP[i]) + PV_INSTALL_FIXCOST * PVDecision[i]



objective += cp.sum(purchasedElectricity) / TypicalDay * 365 * Purchased_Electricity_Rate 

objective -= cp.sum(exportedElectricity) / TypicalDay * 365 * Exported_Electricity_Rate


objective += cp.sum(generationDGNode1) / TypicalDay * 365 * GEN_COST_DG + \
cp.sum(generationDGNode2) / TypicalDay * 365 * GEN_COST_DG + \
cp.sum(generationDGNode3) / TypicalDay * 365 * GEN_COST_DG + \
cp.sum(generationDGNode4) / TypicalDay * 365 * GEN_COST_DG

objective += cp.sum(eleLoaCurNode1) / TypicalDay * 365 * curtailedLoad_Cost + \
cp.sum(eleLoaCurNode2) / TypicalDay * 365 * curtailedLoad_Cost + \
cp.sum(eleLoaCurNode3) / TypicalDay * 365 * curtailedLoad_Cost + \
cp.sum(eleLoaCurNode4) / TypicalDay * 365 * curtailedLoad_Cost

prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver='GUROBI', verbose=True, reoptimize=True)                        
print("status:", prob.status)    
print("optimal value", prob.value) 
print("optimal var", DGNumber.value, batteryCAP.value, PVCAP.value)

end_time = time.time()
print("Time consumed: {:.2f}s".format(end_time - start_time))

In [ ]:
for variable in prob.variables():
        print("Variable %s: value %s" % (variable.name(), variable.value))

In [ ]:
f = open("Results.txt", "w")
f.write("Optimal Cost")
f.write(str(prob.value))
f.write("\n")
for variable in prob.variables():
    f.write(variable.name())
    f.write(str(variable.value))
    f.write("\n")
f.close()

In [ ]:
for i in range(TypicalDay):
    load = (electricalLoad_Node1 + electricalLoad_Node2 + electricalLoad_Node3 + electricalLoad_Node4)[i]
    display(load)
    display(max(load))
    display(load.argmax())

In [ ]:
sec(45*3.14/180)

In [ ]:
math.sqrt(2)

In [ ]:
math.cos(180*3.14/180)

In [ ]:
math.tan(45)

In [ ]:
#     constraints += [VoltageImagPartNode[i] <= K*(VoltageRealPartNode[i]-V_min*sec((Theta_max-Theta_min)/2)*math.cos(Theta_min)) \
#                     + V_min*sec((Theta_max-Theta_min)/2)*math.sin(Theta_min)]
#     constraints += [VoltageImagPartNode[i]<= (math.sin(Theta_max)/(math.cos(Theta_max)-1))*(VoltageRealPartNode[i] - V_max)]
#     constraints += [VoltageImagPartNode[i]<= (-math.sin(Theta_min)/(math.cos(Theta_min)-1))*(VoltageRealPartNode[i] - V_min)]
#     constraints += [VoltageRealPartNode[i]*math.tan(Theta_min)<=VoltageImagPartNode[i],\
#                      VoltageImagPartNode[i]<=VoltageRealPartNode[i]*math.tan(Theta_max)]

In [ ]:
-V_min*sec((Theta_max-Theta_min)/2)*math.cos(Theta_min)

In [ ]:
(-math.sin(Theta_min)/(math.cos(Theta_min)-1))

In [ ]:
math.sin(Theta_max)/(math.cos(Theta_max)-1)

In [ ]:
K

In [ ]:
sec((Theta_max-Theta_min)/2)*math.cos(Theta_min)

In [ ]:
sec((Theta_max-Theta_min)/2)*math.sin(Theta_min)

In [ ]:
math.tan(Theta_min)

In [ ]:
math.tan(Theta_max)

In [ ]:
Vbase**(2)/Sbase


In [ ]:
# Node 1
M = 1
Day = 1 
Pin = injectedActivePowerNode[M].value
Qin = injectedReactivePowerNode[M].value 
PDg = generationDGNode1[Day,:] 
PPv =
PBatt = 



# constraints += [generationPVNode1[i, j] + generationDGNode1[i, j] - (electricalLoad_Node1[i, j] - \
#         eleLoaCurNode1[i, j]) + batDisPowNode1[i, j] - batChaPowNode1[i, j] + injectedPowerNode5_1[i, j] - \
#         injectedPowerNode1_2[i, j] == 0]

In [ ]:
## Plots
# Active Power injection at Nodes
t = np.linspace(0,23,24)
#for day in range(TypicalDay):
day =0
for i in range(NODE):
    Pin = injectedActivePowerNode[i].value
    Qin = injectedReactivePowerNode[i].value
    Pload = 
    plt.step(t, var[day,:], label ="Node"+ str(i+1))
    plt.legend(loc="upper right")
    plt.axis([0, 23, -900, 900])
    plt.margins(0.2)
    plt.xticks(t)
    plt.xlabel('Hour of the day')
    plt.ylabel('Active Power Injected (kW)')
    plt.show()        
        

In [ ]:
injectedReactivePowerNode[4].value

In [ ]:
sec(60*math.pi/180)


In [ ]:
np.array([electricalLoad_Spring_Node1, electricalLoad_Summer_Node1,electricalLoad_Fall_Node1, electricalLoad_Winter_Node1]).shape

In [ ]:
np.ones((1,T)).shape

In [ ]:
Vr = np.zeros(NODE)
Vi = np.zeros(NODE)
P = [1140, -1140, -1000, -1280]
Q = [0, 0, 0, 0]

In [ ]:
for i in range(NODE):
    Vr[i] = (Vo/Vbase) + 1/(Vo/Vbase)*(sum(z_real[i,j]*P[j] + \
                                        z_img[i,j]*Q[j] for j in range(NODE)))/(Zbase*Sbase*1000)
    Vi[i] ==  1/(Vo/Vbase)*(sum(z_img[i,j]*P[j] - \
                             z_real[i,j]*Q[j] for j in range(NODE)))/(Zbase*Sbase*1000)

display(Vr)
display(Vi)

In [ ]:
A = np.zeros(NODE)
B = np.zeros(NODE)
C = np.zeros(NODE)
D = np.zeros(NODE)
E = np.zeros(NODE)

for i in range(NODE):
    A[i] = K*(Vr[i]-V_min*sec((Theta_max-Theta_min)/2)*math.cos(Theta_min)) \
                     + V_min*sec((Theta_max-Theta_min)/2)*math.sin(Theta_min) - Vi[i]
    B[i]  = (math.sin(Theta_max)/(math.cos(Theta_max)-1))*(Vr[i] - V_max) - Vi[i]
    C[i] = (-math.sin(Theta_min)/(math.cos(Theta_min)-1))*(Vr[i] - V_max) - Vi[i]
    D[i] = Vr[i]*math.tan(Theta_min)-Vi[i]
    E[i] = Vi[i]-Vr[i]*math.tan(Theta_max)
    
display(A,B,C,D,E)

In [ ]:
K

In [ ]:
V_min*sec((Theta_max-Theta_min)/2)*math.cos(Theta_min)

In [ ]:
V_min*sec((Theta_max-Theta_min)/2)*math.sin(Theta_min)

In [ ]:
 display(injectedReactivePowerNode[4].value)

In [ ]:
0.4*Ibase*1000*math.sqrt(2)

In [ ]:
f = open("myfile.txt", "x")